In [1]:
import torch
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import pickle
import time
from pathlib import Path

In [2]:
model_path = Path('./TCN/mnist_pixel/models')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

_, test_loader = data_generator(data_path, batch_size)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Torch Model

In [3]:
# Load Model

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, trt=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

if torch.cuda.is_available():
    model.cuda()

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)


In [5]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model(data_queue)
        pred = output.max(1, keepdim=True)[1]
        runtimes.append(time.time() - start)

torch_runtime = np.mean(runtimes)*1000
print(f'Average runtime per sample: {torch_runtime} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 1.6473996091862113 ms


# ONNX GPU Runtime

In [6]:
import onnx
import onnxruntime as ort
from onnx import helper, shape_inference

In [7]:
# Check Model
onnx_model = onnx.load(model_path / (model_name+'.onnx'))
onnx.checker.check_model(onnx_model)
inferred_model = shape_inference.infer_shapes(onnx_model)
onnx.checker.check_model(inferred_model)

# Load Session
onnx_path = model_path / (model_name+'.onnx')
ort_session = ort.InferenceSession(str(onnx_path))

In [8]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        ort_inputs = {ort_session.get_inputs()[0].name: data_queue.cpu().numpy()}
        ort_outs = ort_session.run(None, ort_inputs)
        pred = ort_outs[0].argmax(axis=1)[0]
        runtimes.append(time.time() - start)

onnx_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {onnx_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.5486956056283444 ms


# TensorRT Model

In [9]:
from torch2trt import TRTModule

/usr/local/lib/python3.8/dist-packages/torch2trt-0.2.0-py3.8-linux-x86_64.egg/torch2trt/converters/dummy_converters.py:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return method[0] == '_' and method[1] is not '_'


In [10]:
# Load Model
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(model_path / (model_name+'_trt.pt')))

<All keys matched successfully>

In [11]:
num_samples = 10
runtimes = []

data_queue = torch.zeros((1,1,28*28))
data_queue = data_queue.cuda() if torch.cuda.is_available() else data_queue

for s, (data, target) in enumerate(test_loader):
    if s >= num_samples:
        break
    print(f'Running sample image {s+1}')
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        start = time.time()
        output = model_trt(data_queue)
        pred = output.max(2, keepdim=True)[1]
        runtimes.append(time.time() - start)

tensorrt_runtimes = np.mean(runtimes)*1000
print(f'Average runtime per sample: {tensorrt_runtimes} ms')

Running sample image 1
Running sample image 2
Running sample image 3
Running sample image 4
Running sample image 5
Running sample image 6
Running sample image 7
Running sample image 8
Running sample image 9
Running sample image 10
Average runtime per sample: 0.5026635466789713 ms


In [12]:
print(f'ONNX Runtime is {torch_runtime/onnx_runtimes} times faster than PyTorch (gpu)')
print(f'TensorRT Runtime is {torch_runtime/tensorrt_runtimes} times faster than regular PyTorch')

ONNX Runtime is 3.002392569373095 times faster than PyTorch (gpu)
TensorRT Runtime is 3.277340519459494 times faster than regular PyTorch


In [15]:
model_engine = model_trt.engine
dir(model_engine)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'binding_is_input',
 'create_execution_context',
 'create_execution_context_without_device_memory',
 'device_memory_size',
 'get_binding_bytes_per_component',
 'get_binding_components_per_element',
 'get_binding_dtype',
 'get_binding_format',
 'get_binding_format_desc',
 'get_binding_index',
 'get_binding_name',
 'get_binding_shape',
 'get_binding_vectorized_dim',
 'get_location',
 'get_profile_shape',
 'get_profile_shape_input',
 'has_implicit_batch_dimension',
 'is_execution_binding',
 'is_shape_binding',
 'max_batch_size',
 'max_workspace_size',
 'name',
 'num_bindings',
 'num_laye

In [31]:
model_engine.get_binding_shape(1)

(1, 10)

In [19]:
with open(model_path / (model_name+'.engine'), 'wb') as f:
    f.write(model_trt.engine.serialize())